In [1]:
import pandas as pd
import numpy as np
import zipfile
z = zipfile.ZipFile('1.kc_house_data.csv.zip')
sales = pd.read_csv(z.open('kc_house_data.csv'))
z = zipfile.ZipFile('1.kc_house_train_data.csv.zip')
train_data = pd.read_csv(z.open('kc_house_train_data.csv'))
z = zipfile.ZipFile('1.kc_house_test_data.csv.zip')
test_data = pd.read_csv(z.open('kc_house_test_data.csv'))
print sales.shape
print train_data.shape
print test_data.shape

(21613, 21)
(17384, 21)
(4229, 21)


In [3]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
from sklearn.linear_model import LinearRegression
example_model = LinearRegression()
example_model.fit(train_data[example_features], train_data['price'])
print example_model.coef_
print example_model.intercept_

[   315.40669062 -65081.88711588   6942.16598637]
87912.865815


In [5]:
example_predictions = example_model.predict(train_data[example_features])
print example_predictions[0]

271789.26538


In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    errors = outcome-predictions
    # Then square and add them up
    RSS = (errors*errors).sum()
    return(RSS) 

In [9]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761940583e+14


In [14]:
from math import log
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']*test_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data['lat']+train_data['long']
test_data['lat_plus_long'] = test_data['lat']+test_data['long']

In [20]:
#Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)
print test_data['bedrooms_squared'].mean()
print test_data['bed_bath_rooms'].mean()
print test_data['log_sqft_living'].mean()
print test_data['lat_plus_long'].mean()

12.4466777016
7.50390163159
7.55027467965
-74.653333554


In [21]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [29]:
model_1 = LinearRegression()
model_1.fit(train_data[model_1_features], train_data['price'])
model_2 = LinearRegression()
model_2.fit(train_data[model_2_features], train_data['price'])
model_3 = LinearRegression()
model_3.fit(train_data[model_3_features], train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [30]:
#Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?
#Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?
print model_1.coef_
print model_2.coef_

[  3.12258646e+02  -5.95865332e+04   1.57067421e+04   6.58619264e+05
  -3.09374351e+05]
[  3.06610053e+02  -1.13446368e+05  -7.14613083e+04   6.54844630e+05
  -2.94298969e+05   2.55796520e+04]


In [31]:
#Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data? Is this what you expected?
print get_residual_sum_of_squares(model_1, train_data[model_1_features], train_data['price'])
print get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data['price'])
print get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data['price'])

9.6787996305e+14
9.58419635074e+14
9.0343645505e+14


In [33]:
#Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data? Is this what you expected?Think about the features that were added to each model from the previous.
print get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data['price'])
print get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price'])
print get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data['price'])

2.25500469795e+14
2.23377462976e+14
2.59236319207e+14
